In [2]:
!ls

Untitled.ipynb	   dataset-food17      result_svm.csv  tutorials
data_explor.ipynb  foodclassify.ipynb  src


In [4]:
import torch
from torch.utils.data import DataLoader as dalo
from torch.utils.data import random_split
from torch.utils.data import Dataset
import torchvision.datasets as datasets
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models
from sklearn.model_selection import GridSearchCV

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
resnet34 = models.resnet34(pretrained=True).to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/jupyter/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
      super(CNN, self).__init__()
      self.resnet34 = models.resnet34(pretrained=True).to(device)
      self.conv1 = self.resnet34.conv1
      self.bn1 = self.resnet34.bn1
      self.relu = self.resnet34.relu
      self.maxpool = self.resnet34.maxpool
      self.layer1 = self.resnet34.layer1
      self.layer2 = self.resnet34.layer2
      self.layer3 = self.resnet34.layer3
      self.layer4 = self.resnet34.layer4
      self.avgpool = self.resnet34.avgpool
      self.fc1 = torch.nn.Linear(128,17, bias=False)
  
  def forward(self, x):
      relu = torch.nn.ReLU()
      softmax = torch.nn.Softmax(dim=1)
      x = x.to(device)
      x = self.conv1(x)
      x = self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)
      x = self.layer1(x)
      x = self.layer2(x)
      x = self.avgpool(x)
      x = torch.flatten(x, 1)
      x = softmax(self.fc1(x))
      return x

def load_data(path='', batch=30, transform=None):
    train_dataset = datasets.ImageFolder(path+'/train', transform=transform)
    test_dataset = datasets.ImageFolder(path+'/test', transform=transform)
    n_train = round(len(train_dataset)*0.8)
    train_data, valid_data = random_split(train_dataset, [n_train, len(train_dataset)-n_train])
    train_set = dalo(train_data, batch_size=batch, shuffle=True)
    valid_set = dalo(valid_data)
    test_set = dalo(test_dataset)
    return train_set, valid_set, test_set

def train_(model, train_set, valid_set,epochs, model_num, lr, b):
  _, ((ax1), (ax2)) = plt.subplots(nrows=1,ncols=2)
  train_loss_tmp = []
  valid_loss_tmp = []
  accuracy_valid_tmp = []
  accuracy_train_tmp = []
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  valid_loss_min = np.Inf
  print('##########  START TRAINING  ##########')
  for epoch in range(epochs):
    model.train()
    train_size = 0
    train_loss = 0
    correct_train = 0
    for data, label in train_set:
      train_size += data.shape[0]
      data = data.type(torch.float32).to(device)
      label = label.type(torch.float32).to(device)
      optimizer.zero_grad()
      output = model(data)
      _, predict = output.max(1)
      correct_train += predict.eq(label).sum().item()
      loss = criterion(output, label.long())
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
    accuracy_train_tmp.append(100.*correct_train/train_size)
    model.eval()
    valid_loss = 0
    with torch.no_grad():
      valid_size = 0
      correct_valid = 0
      for data, label in valid_set:
        valid_size += data.shape[0]
        data = data.type(torch.float32).to(device)
        label = label.type(torch.float32).to(device)
        output = model(data)
        _, predict = output.max(1)
        #print('predict{} -- label{}'(predict))
        correct_valid += predict.eq(label).sum().item()
        loss = criterion(output, label.long())
        valid_loss += loss.item()
        # print('train_loss', train_loss)
        # print('valid loss', valid_loss)
      train_loss = train_loss/train_size
      valid_loss = valid_loss/valid_size
      valid_loss_tmp.append(valid_loss)
      train_loss_tmp.append(train_loss)
      accuracy_valid_tmp.append(100.*correct_valid/valid_size)
      model_save = 'model_e'+str(epochs)+'_b'+str(b)+'_lr'+str(lr)+'.pt'
      print('{} from {} || Training Loss: {:.6} || Valid Loss: {:.6} || train Accuracy: {:.6}% || Valid Accuracy: {:.6}%'\
            .format(epoch+1, epochs, train_loss, valid_loss, accuracy_train_tmp[-1], accuracy_valid_tmp[-1]))
      if valid_loss <= valid_loss_min:
        print('**Valid loss decrease from {:.6} --> {:.6}. Saving model ...'.format(valid_loss_min, valid_loss))
        torch.save(model.state_dict(), model_save)
        valid_loss_min = valid_loss

  ax1.plot(valid_loss_tmp, label='Valid loss')
  ax1.plot(train_loss_tmp, label='train loss')
  ax2.plot(accuracy_valid_tmp, label='accuracy valid')
  ax2.plot(accuracy_train_tmp, label='accuracy train')
  ax2.plot()
  plt.legend()
  plt.title('TH FOOD17'+'model_'+'b'+str(b)+'_lr'+str(lr)+'.pt')
  plt.show()
  print('########## FINISH TRAINING ##########')

def test_(model, test_set):
  modelpt = ['./model_e140_b20_lr0.001.pt']
  for i in modelpt:
    model.load_state_dict(torch.load(i))
    accuracy_=0
    correct=0
    test_size = len(test_set)
    with torch.no_grad():
      for data, label in test_set:
        data = data.type(torch.float32).to(device)
        label = label.type(torch.float32).to(device)
        output = model(data)
        _, predict = output.max(1)
        #print('predict{} -- label{}'(predict))
        correct+= predict.eq(label).sum().item()
      accuracy_ = 100.*correct/test_size
      print(i[2:],' accuracy=', accuracy_)


if __name__ ==  '__main__':
  batch=[10,20,30]
  epochs = [80,100,120,140]
  lr = [0.001, 0.00001]
  tran = transforms.Compose( [transforms.Resize((256, 256)),
                      transforms.ToTensor(),
                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225])])
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model_num = 0
  for b in batch:
    for e in epochs:
      for l in lr:
        model_num += 1
        train_dataset, valid_dataset, _ = load_data('./dataset-food17',b,transform=tran)
        torch.cuda.empty_cache()
        model = CNN().to(device)
        train_(model, train_dataset, valid_dataset, e, model_num, l, b)


##########  START TRAINING  ##########
1 from 80 || Training Loss: 0.136844 || Valid Loss: 2.68926 || train Accuracy: 22.1504% || Valid Accuracy: 23.1018%
**Valid loss decrease from inf --> 2.68926. Saving model ...
2 from 80 || Training Loss: 0.133103 || Valid Loss: 2.68002 || train Accuracy: 27.4859% || Valid Accuracy: 24.3942%
**Valid loss decrease from 2.68926 --> 2.68002. Saving model ...
3 from 80 || Training Loss: 0.130586 || Valid Loss: 2.57864 || train Accuracy: 34.0744% || Valid Accuracy: 36.0258%
**Valid loss decrease from 2.68002 --> 2.57864. Saving model ...
4 from 80 || Training Loss: 0.128135 || Valid Loss: 2.57319 || train Accuracy: 40.0162% || Valid Accuracy: 38.4491%
**Valid loss decrease from 2.57864 --> 2.57319. Saving model ...
5 from 80 || Training Loss: 0.126747 || Valid Loss: 2.54407 || train Accuracy: 42.3605% || Valid Accuracy: 39.2569%
**Valid loss decrease from 2.57319 --> 2.54407. Saving model ...
6 from 80 || Training Loss: 0.125136 || Valid Loss: 2.51605 